## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

170500096/170498071 [==============================] - 162s 1us/step
(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder(categories='auto')
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [4]:
classifier=Sequential()

#卷積組合
#'自己設計參數' eg:32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(Convolution2D(32, (3, 3), input_shape=(32, 32, 3), activation='relu'))
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(64, (3, 3), input_shape=(32, 32, 3), activation='relu')) #'自己設計參數'
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
#'自己設計FC層參數'
classifier.add(Dense(output_dim=100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10, activation='softmax')) #'輸出函數應該用什麼？'

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100)`
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
50000/50000 [==============================] - 43s 851us/step - loss: 1.3781 - acc: 0.5530
Epoch 2/100
50000/50000 [==============================] - 41s 810us/step - loss: 0.7816 - acc: 0.7256
Epoch 3/100
50000/50000 [==============================] - 40s 809us/step - loss: 0.5286 - acc: 0.8149
Epoch 4/100
50000/50000 [==============================] - 40s 807us/step - loss: 0.3329 - acc: 0.8868
Epoch 5/100
50000/50000 [==============================] - 41s 816us/step - loss: 0.2149 - acc: 0.9266
Epoch 6/100
50000/50000 [==============================] - 40s 808us/step - loss: 0.1546 - acc: 0.9470
Epoch 7/100
50000/50000 [==============================] - 40s 803us/step - loss: 0.1294 - acc: 0.9556
Epoch 8/100
50000/50000 [==============================] - 40s 808us/step - loss: 0.1252 - acc: 0.9578
Epoch 9/100
50000/50000 [==============================] - 40s 801us/step - loss

50000/50000 [==============================] - 39s 774us/step - loss: 0.0153 - acc: 0.9955
Epoch 74/100
50000/50000 [==============================] - 39s 775us/step - loss: 0.0172 - acc: 0.9950
Epoch 75/100
50000/50000 [==============================] - 39s 775us/step - loss: 0.0120 - acc: 0.9965
Epoch 76/100
50000/50000 [==============================] - 39s 775us/step - loss: 0.0135 - acc: 0.9959
Epoch 77/100
50000/50000 [==============================] - 42s 840us/step - loss: 0.0156 - acc: 0.9953
Epoch 78/100
50000/50000 [==============================] - 41s 812us/step - loss: 0.0165 - acc: 0.9951
Epoch 79/100
50000/50000 [==============================] - 40s 807us/step - loss: 0.0107 - acc: 0.9967
Epoch 80/100
50000/50000 [==============================] - 40s 805us/step - loss: 0.0115 - acc: 0.9966
Epoch 81/100
50000/50000 [==============================] - 40s 800us/step - loss: 0.0127 - acc: 0.9962
Epoch 82/100
50000/50000 [==============================] - 40s 810us/step - 

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[1.14164486e-07, 9.96446659e-09, 9.70711705e-11, 9.99807060e-01,
        1.26527812e-04, 1.74776038e-09, 5.38780057e-07, 6.57595592e-05,
        7.70739104e-13, 3.26683117e-15]], dtype=float32)